In [1]:
import wordcloud

In [2]:
# Importation des packages nécessaires
import numpy as np
import nltk
import os
import nltk.corpus
import re
import functools
import operator
import datetime as dt
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


from matplotlib import pyplot as plt 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Achar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Achar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Achar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Création du dataFrame contenant les données de Joe Biden
df_biden = pd.read_csv('hashtag_joebiden.csv', lineterminator='\n')
df_biden['mention'] = 'Biden'

# Création du DataFrame contenant les données de Donald Trump
df_trump = pd.read_csv('hashtag_donaldtrump.csv', lineterminator='\n')
df_trump['mention'] = 'Trump'

# Concaténation des deux DataFrames en un seul
df_full = [df_trump,df_biden]
df_full = pd.concat(df_full)

# Identification des tweets mentionnant les deux candidats (duplicatas)
df_full['duplicated'] = df_full.duplicated(subset='tweet', keep=False)
df_full.loc[(df_full['duplicated'] == True),'mention'] = 'Trump & Biden'

#conversion des dates en datetime
df_full['created_at'] = pd.to_datetime(df_full["created_at"])
df_full.sort_values('created_at', inplace=True)

#Retrait des heures pour pouvoir regrouper les dates plus facilement
df_full['created_at'] = pd.to_datetime(df_full["created_at"]).dt.date

#Conversion des likes en int
df_full['likes'] = df_full['likes'].astype(int)

# Suppression des doublons
df_full = df_full.drop_duplicates(subset='tweet').reset_index(drop=True)

# Modification du Dataframe des candidats pour ne contenir que les tweets mentionnant uniquement ceux-ci
df_trump = df_full.loc[df_full['mention'] == 'Trump']
df_biden = df_full.loc[df_full['mention'] == 'Biden']

# Compte par continent

In [ ]:
countplot = sns.catplot(kind='count',x='continent', data=df_full.sort_values('mention'), hue='mention')
countplot.set_xticklabels(rotation=80)
colors = ['#258BF3','#25F347','#F1120B']
sns.set_palette(sns.color_palette(colors))
countplot

# Mots préférés

In [ ]:
# Déclaration d'une fonction pour nettoyer le texte des tweets
def  clean_text(df, text_field, new_text_field_name):
    
    df[new_text_field_name] = df[text_field].str.lower()
    
    # Nettoyage du texte des tweets
    # Remplacement par '' de toutes les mentions @xyz, sites webs et caractères non compris entre a-z ou A-Z ou 0-9
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    
    # Remplacement par '' de tous les nombres
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    
    return df

# Définition d'une fonction pour lemmatiser le texte (trouver la racine des mots)
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

# Définition d'une fonction pour afficher le word cloud en fonction des paramètres choisis
def plot_cloud(wordcloud):
    plt.figure(figsize=(5,5))
    plt.imshow(wordcloud)
    plt.axis("off")

## 1. Trump

In [ ]:
# Création d'un dataframe ne contenant que les tweets de Trump provenants des États-Unis
trump_tweet_usa = df_trump[df_trump['country'] == 'United States of America'][['tweet']]

# Nettoyage des tweets de Trump aux USA et affichage des premières lignes
data_clean = clean_text(trump_tweet_usa, 'tweet', 'tweet_clean')
data_clean.head()

In [ ]:
# Création d'une variable qui contient les 'stopwords' anglais, 
# c-à-d les mots non-essentiels, qui seront à retirer (you, a, I, etc...)
from nltk.corpus import stopwords
stop = stopwords.words('english')

# Retrait de tous les stopwords et ajout dans une nouvelle série sous le nom tweet_clean
data_clean['tweet_clean'] = data_clean['tweet_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data_clean.head()

In [ ]:
# Création d'une liste de tokens à évaluer pour chaque tweet (découpage de chaque mot) 
# et ajout au df sous le nom tweet_tokens
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

data_clean['tweet_tokens'] = data_clean['tweet_clean'].apply(lambda x: word_tokenize(x))
data_clean.head()

In [ ]:
# Lemmatisation des tweets et ajout au df sous le nom tweet_tokens_lemma
from nltk.stem import WordNetLemmatizer
data_clean['tweet_tokens_lemma'] = data_clean['tweet_tokens'].apply(lambda x: word_lemmatizer(x))
data_clean.head()

In [ ]:
from wordcloud import WordCloud
from PIL import Image
import numpy as np

# Transformation des tweet_tokens_lemma (liste de listes) en liste applatie
trump_token_list = functools.reduce(operator.iconcat, data_clean['tweet_tokens_lemma'].values.tolist(), [])

# Transformation de la liste applatie en string
trump_token_string = ' '.join(trump_token_list)

# Choix du masque à utiliser pour Trump
mask_trump = np.array(Image.open('donaldtrump_mask.png'))

# Définition des paramètres du wordcloud
wordcloud_trump = WordCloud(background_color = 'black', random_state=1,
                      colormap='Reds', collocations=False,
                      mask=mask_trump, contour_color='white').generate(trump_token_string)

# Traçage du word cloud et aperçu de l'image
trump_plot = plot_cloud(wordcloud_trump)
trump_plot

## 2. Biden

In [ ]:
# Création d'un dataframe ne contenant que les tweets de TBiden provenants des États-Unis
biden_tweet_usa = df_biden[df_biden['country'] == 'United States of America'][['tweet']]

# Nettoyage des tweets de Biden aux USA à l'aide de la fonction précédemment définie
data_clean = clean_text(biden_tweet_usa, 'tweet', 'tweet_clean')

# Retrait de tous les stopwords et ajout dans une nouvelle série sous le nom tweet_clean
data_clean['tweet_clean'] = data_clean['tweet_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# Création d'une liste de tokens à évaluer pour chaque tweet (découpage de chaque mot) 
# et ajout au df sous le nom tweet_tokens
data_clean['tweet_tokens'] = data_clean['tweet_clean'].apply(lambda x: word_tokenize(x))

# Lemmatisation des tweets et ajout au df sous le nom tweet_tokens_lemma
data_clean['tweet_tokens_lemma'] = data_clean['tweet_tokens'].apply(lambda x: word_lemmatizer(x))

# Transformation des tweet_tokens_lemma (liste de listes) en liste applatie
biden_token_list = functools.reduce(operator.iconcat, data_clean['tweet_tokens_lemma'].values.tolist(), [])

# Transformation de la liste applatie en string
biden_token_string = ' '.join(biden_token_list)

# Choix du masque à utiliser pour Biden
mask_biden = np.array(Image.open('joebiden_mask.png'))

# Définition des paramètres du wordcloud
wordcloud_biden = WordCloud(background_color = 'black', random_state=1,
                      colormap='Blues', collocations=False,
                      mask=mask_biden, contour_color='white').generate(biden_token_string)

# Traçage du word cloud
biden_plot = plot_cloud(wordcloud_biden)
biden_plot

## 3. Visualisation côte-à-côte

In [ ]:
# Création d'une figure pour contenir les 2 images
fig = plt.figure(figsize=(20,20), facecolor = 'black')

# Paramétrage de la première sous-figure
ax1 = fig.add_subplot(1,2,1)
ax1.axis("off")
ax1.imshow(wordcloud_biden)

# Paramétrage de la deuxième sous-figure
ax2 = fig.add_subplot(1,2,2)
ax2.axis("off")
ax2.imshow(wordcloud_trump)

# Nombre de tweets par jour

In [ ]:
countplot = sns.catplot(kind='count',y='created_at', data=df_full.sort_values('mention'), hue='mention')
countplot.set_xticklabels(rotation=80)
colors = ['#258BF3','#F1120B','#25F347']
sns.set_palette(sns.color_palette(colors))
countplot

# Nombre de like par jour

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mpl_dates
# Création d'un dataframe comptant le nombre de likes par jour pour chaque candidat à partir du dataframe complet
df_graph_t = pd.DataFrame(df_full.loc[df_full['mention'] == 'Trump'].groupby('created_at')['likes'].sum())
df_graph_b = pd.DataFrame(df_full.loc[df_full['mention'] == 'Biden'].groupby('created_at')['likes'].sum())
df_graph_bt = pd.DataFrame(df_full.loc[df_full['mention'] == 'Trump & Biden'].groupby('created_at')['likes'].sum())

# Jointure des dataframes
df_graph = df_graph_t.join(df_graph_b, how='outer', on='created_at', lsuffix = '_trump', rsuffix = '_biden')
df_graph = df_graph.join(df_graph_bt, how='outer', on='created_at', rsuffix = '_trump & biden')

# Traçage du graphique
date_format= mpl_dates.DateFormatter('%d/%m')
plt.style.use('seaborn-v0_8')
plt.figure(figsize=(10,6))
plt.plot(df_graph.likes_trump / 1000, marker ='o',lw =2,linestyle='solid',markersize= 10, label = 'Trump', color = 'r')
plt.plot(df_graph.likes_biden / 1000, marker ='o',lw =2,linestyle='solid',markersize= 10, color = 'b', label = 'Biden')
plt.plot(df_graph.likes / 1000, marker ='o',lw =2,linestyle='solid',markersize= 10, color = 'g', label = 'Trump & Biden')
plt.legend()
plt.xlabel('date')
plt.ylabel('Nombre (k)')
plt.title('Nombre de likes sur les Tweets mentionnant les candidats, selon le jour')
plt.tight_layout()
plt.gca().xaxis.set_major_formatter(date_format)
plt.show

# Plateformes utilisées

In [ ]:
# Création d'un nouveau dataframe ne contenant que les sources les plus populaires pour les Tweets
df_sources = df_full[(df_full['source'] == 'Twitter for iPhone') | (df_full['source'] == 'Twitter for Android') 
                     | (df_full['source'] == 'Twitter Web App')].sort_values('mention')

# Traçage du graphique
countplot = sns.catplot(kind='count',x='source', data=df_sources, hue='mention')
countplot.set_xticklabels(rotation = 80)
colors = ['#258BF3','#25F347','#F1120B']
sns.set_palette(sns.color_palette(colors))
countplot

# Nombre de likes sur les tweets par état américain

## 1. Trump

In [ ]:
import plotly.graph_objects as go
# Création d'un dataframe ne contenant que les tweets localisés aux États-Unis
dtus= df_trump[df_trump.country == "United States of America"]

# Filtrage des colonnes nécessaires
dtus = dtus[['likes','retweet_count','state','state_code','country']]

# Regroupement des likes par état puis conversion en dataframe car le resultat sortant est une série
dtus_grouped = dtus.groupby('state_code')['likes'].sum()
dtfinal = dtus_grouped.to_frame().reset_index()


#Création de la carte choropleth
fig = go.Figure(data=go.Choropleth(
    locations=dtfinal['state_code'], # coordonnées spatiales
    z = dtfinal['likes'].astype(float), # On associe la couleur au nombre de likes
    locationmode = 'USA-states', 
    colorscale = 'turbo',
    autocolorscale=False,
    colorbar_title = "Nombre de likes",
    zmin = 0,
    zmax = 700000,
    marker_line_color='white'
))

fig.update_layout(
    title_text = 'Nombre de like par état pour Trump',
    geo_scope='usa', # centré sur les USA
)

fig.show()

## 2. Biden

In [ ]:
# Création d'un dataframe ne contenant que les tweets localisés aux États-Unis
dbus = df_biden[df_biden.country=="United States of America"]

# Filtrage des colonnes nécessaires
dbus = dbus[['likes','retweet_count','state','state_code','country']]

# Regroupement des likes par état puis conversion en dataframe car le resultat sortant est une série
dbus_grouped = dbus.groupby('state_code')['likes'].sum()
dbfinal = dbus_grouped.to_frame().reset_index()

#Création de la carte choropleth
fig = go.Figure(data=go.Choropleth(
    locations=dbfinal['state_code'], # coordonnées spatiales
    z = dbfinal['likes'].astype(float), # On associe la couleur au nombre de likes
    locationmode = 'USA-states', 
    colorscale = 'turbo',
    autocolorscale=False,
    colorbar_title = "Nombre de likes",
    zmin = 0,
    zmax = 700000,
    marker_line_color='white'
))

fig.update_layout(
    title_text = 'Nombre de like par état pour Biden',
    geo_scope='usa', # centré sur les USA
)

fig.show()

## 3. Candidat ayant le plus de likes par état

In [ ]:
# Création d'un dataframe contenant le nombre de likes pour chaque candidat selon l'État
df_likesperstate = dbfinal.join(dtfinal.set_index('state_code'),on='state_code', 
                                how='outer', lsuffix = '_biden',rsuffix='_trump')

# Ajout d'une Série contenant le le candidat ayant le plus de likes pour chaque État
df_likesperstate['gagnant'] = np.where(df_likesperstate['likes_trump'] > df_likesperstate['likes_biden'], 'Trump', 'Biden')

# Traçage de la carte Choropleth
fig = go.Figure(data=px.choropleth(
    locations=df_likesperstate['state_code'], # coordonnées spatiales
    color = df_likesperstate['gagnant'], # On associe la couleur au nombre de likes
    locationmode = 'USA-states',
    hover_data = ['likes_trump', 'likes_biden'],
    data_frame = df_likesperstate
))

fig.update_layout(
    title_text = 'Gagnant par état selon le nombre de likes sur Twitter',
    geo_scope='usa', # centré sur les USA
)

fig.show()